In [ ]:
# 1. Proje Kurulumu ve Güncelleme
import os
import sys

# Google Colab kontrolü
try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("Google Colab ortamı algılandı.")
    repo_url = "https://github.com/onndd/newproje.git"
    project_dir = "/content/newproje"
    
    if os.path.exists(project_dir):
        print("Proje klasörü mevcut. Güncelleniyor...")
        %cd {project_dir}
        !git pull
    else:
        print("Proje klonlanıyor...")
        !git clone {repo_url}
        %cd {project_dir}
        
    # Gerekli kütüphaneler
    print("Kütüphaneler yükleniyor...")
    %pip install catboost hmmlearn tensorflow lightgbm scikit-learn
    
    # Path ayarı
    sys.path.append(project_dir)
else:
    print("Yerel ortam algılandı.")
    project_dir = os.getcwd()
    sys.path.append(project_dir)

In [ ]:
# 2. Veri Yükleme
import sqlite3
import pandas as pd
import numpy as np
from jetx_project.data_loader import load_data
from jetx_project.config import DB_PATH

print("Veri yükleniyor...")
try:
    df = load_data(DB_PATH)
    print(f"Toplam {len(df)} veri yüklendi.")
    print(df.head())
except Exception as e:
    print(f"Hata: {e}")
    print("Lütfen jetx.db dosyasının proje klasöründe olduğundan emin olun.")

In [ ]:
# 3. Psikolojik Modeller (HMM) Eğitimi
print("Psikolojik Modeller (HMM) eğitiliyor...")
from jetx_project.model_hmm import train_hmm_model, predict_hmm_state, save_hmm_model

if 'df' in locals() and len(df) > 0:
    # HMM Eğitimi (Gözetimsiz)
    # Log-Transform kullanılarak eğitiliyor (model_hmm.py içinde güncellendi)
    hmm_model, hmm_map = train_hmm_model(df['value'].values)
    save_hmm_model(hmm_model, hmm_map, output_dir='models')
    
    # Tüm geçmiş için durum tahmini (Feature olarak kullanmak için)
    # predict fonksiyonu da artık log-transform kullanıyor
    hmm_states_mapped = predict_hmm_state(hmm_model, df['value'].values, hmm_map)
    
    print("HMM Durumları belirlendi.")
    print(f"Durum Dağılımı: {np.bincount(hmm_states_mapped)}")
else:
    print("Veri yok, HMM eğitilemedi.")

In [ ]:
# 4. Model A (CatBoost) Eğitimi
from jetx_project.model_a import prepare_model_a_data, train_model_a, save_models

if 'df' in locals() and len(df) > 500:
    print("Model A için veri hazırlanıyor...")
    X, y_p15, y_p3, y_x = prepare_model_a_data(df['value'].values, hmm_states_mapped)
    
    print(f"Feature seti boyutu: {X.shape}")
    print("Model A (CatBoost) eğitiliyor...")
    modelA_p15, modelA_p3, modelA_x = train_model_a(X, y_p15, y_p3, y_x)
    save_models(modelA_p15, modelA_p3, modelA_x, output_dir='models')
    print("Model A tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 5. Model B (k-NN) Eğitimi
from jetx_project.model_b import build_memory, train_model_b, save_memory

if 'df' in locals() and len(df) > 500:
    print("Model B (Hafıza) oluşturuluyor...")
    patterns, targets = build_memory(df['value'].values)
    nbrs = train_model_b(patterns)
    save_memory(nbrs, patterns, targets, output_dir='models')
    print("Model B tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 6. Model C (LSTM) Eğitimi
from jetx_project.model_lstm import train_model_lstm, save_lstm_models

if 'df' in locals() and len(df) > 500:
    print("Model C (LSTM) eğitiliyor...")
    # LSTM için veriyi kendi içinde hazırlar
    modelC_p15, modelC_p3, scaler_lstm = train_model_lstm(df['value'].values)
    save_lstm_models(modelC_p15, modelC_p3, scaler_lstm, output_dir='models')
    print("Model C tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 7. Model D (LightGBM) Eğitimi
from jetx_project.model_lightgbm import train_model_lightgbm, save_lightgbm_models

if 'df' in locals() and len(df) > 500:
    print("Model D (LightGBM) eğitiliyor...")
    # Model A ile aynı feature setini kullanır
    modelD_p15, modelD_p3 = train_model_lightgbm(X, y_p15, y_p3)
    save_lightgbm_models(modelD_p15, modelD_p3, output_dir='models')
    print("Model D tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 8. Model E (MLP) Eğitimi
from jetx_project.model_mlp import train_model_mlp, save_mlp_models

if 'df' in locals() and len(df) > 500:
    print("Model E (MLP) eğitiliyor...")
    # Model A ile aynı feature setini kullanır ama içinde filtreler
    modelE_p15, modelE_p3, mlp_cols = train_model_mlp(X, y_p15, y_p3)
    save_mlp_models(modelE_p15, modelE_p3, mlp_cols, output_dir='models')
    print("Model E tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 9. BÜYÜK FİNAL: Simülasyon ve Karşılaştırmalı Rapor
import matplotlib.pyplot as plt
from jetx_project.simulation import run_simulation

if 'df' in locals() and len(df) > 2000:
    print("--- BÜYÜK FİNAL: Simülasyon Başlıyor ---")
    
    test_size = 2000
    test_df = df.iloc[-test_size:].copy()
    
    # --- Model A (CatBoost) Tahminleri ---
    print("Model A tahmin ediyor...")
    X_test_a, _, _, _ = prepare_model_a_data(df['value'].values, hmm_states_mapped, start_index=len(df)-test_size)
    # X_test_a boyutu test_size-1 olabilir, hizalama yapalım
    # prepare_model_a_data N-1 döndürür. test_df'in son satırı hariç alalım.
    sim_df = test_df.iloc[:-1].copy()
    
    sim_df['prob_A_1.5'] = modelA_p15.predict_proba(X_test_a)[:, 1]
    sim_df['prob_A_3.0'] = modelA_p3.predict_proba(X_test_a)[:, 1]
    
    # --- Model C (LSTM) Tahminleri ---
    print("Model C (LSTM) tahmin ediyor...")
    from jetx_project.model_lstm import create_sequences
    # LSTM için veriyi hazırla
    # Dikkat: LSTM sequence length kadar veriyi yutar. 
    # Test verisinin başından seq_length kadar öncesini de almalıyız ki tam test_size kadar tahmin yapabilelim.
    seq_len = 200
    raw_values = df['value'].values
    # Test başlangıcından seq_len öncesinden başla
    start_idx = len(df) - test_size - seq_len
    test_values_extended = raw_values[start_idx:]
    
    # Scale
    test_values_scaled = scaler_lstm.transform(test_values_extended.reshape(-1, 1))
    X_lstm, _, _, _ = create_sequences(test_values_scaled, seq_len)
    
    # X_lstm boyutu tam test_size olmalı
    # create_sequences len(values) - seq_len kadar üretir.
    # len(test_values_extended) = test_size + seq_len
    # output = test_size. Tamam.
    
    # Ancak sim_df bir eksik (N-1). LSTM sonuncuyu da tahmin eder. 
    # Hizalamak için son tahmini atacağız.
    preds_c_p15 = modelC_p15.predict(X_lstm).flatten()
    preds_c_p3 = modelC_p3.predict(X_lstm).flatten()
    
    sim_df['prob_C_1.5'] = preds_c_p15[:-1]
    sim_df['prob_C_3.0'] = preds_c_p3[:-1]
    
    # --- Model D (LightGBM) Tahminleri ---
    print("Model D (LightGBM) tahmin ediyor...")
    sim_df['prob_D_1.5'] = modelD_p15.predict_proba(X_test_a)[:, 1]
    sim_df['prob_D_3.0'] = modelD_p3.predict_proba(X_test_a)[:, 1]
    
    # --- Model E (MLP) Tahminleri ---
    print("Model E (MLP) tahmin ediyor...")
    # MLP için sadece raw featureları seç
    X_test_mlp = X_test_a[mlp_cols]
    sim_df['prob_E_1.5'] = modelE_p15.predict_proba(X_test_mlp)[:, 1]
    sim_df['prob_E_3.0'] = modelE_p3.predict_proba(X_test_mlp)[:, 1]
    
    # --- SİMÜLASYON KOŞUSU ---
    
    # Geçici fonksiyon: Tek model için simülasyon wrapper
    def run_single_model_sim(name, prob_col_15, prob_col_3):
        temp_df = sim_df.copy()
        # Simülasyon fonksiyonu 'prob_1.5' ve 'prob_3.0' sütunlarını bekler
        temp_df['prob_1.5'] = temp_df[prob_col_15]
        temp_df['prob_3.0'] = temp_df[prob_col_3]
        # Pred value şimdilik boş, sadece olasılık bazlı kasa yönetimi test edelim
        temp_df['pred_value'] = 0 
        
        print(f"\n>>> {name} Simülasyonu <<<")
        run_simulation(temp_df, model_name=name)
        
    run_single_model_sim("Model A (CatBoost)", 'prob_A_1.5', 'prob_A_3.0')
    run_single_model_sim("Model C (LSTM)", 'prob_C_1.5', 'prob_C_3.0')
    run_single_model_sim("Model D (LightGBM)", 'prob_D_1.5', 'prob_D_3.0')
    run_single_model_sim("Model E (MLP)", 'prob_E_1.5', 'prob_E_3.0')
    
    # --- GÜVEN DAĞILIM GRAFİKLERİ ---
    print("\n--- Model Güven Dağılımları Hazırlanıyor ---")
    plt.figure(figsize=(15, 10))
    
    # 1.5x Olasılıkları
    plt.subplot(2, 1, 1)
    plt.title("1.5x Üstü Olasılık Dağılımı (Modeller Ne Kadar Emin?)")
    plt.hist(sim_df['prob_A_1.5'], bins=50, alpha=0.5, label='Model A (CatBoost)')
    plt.hist(sim_df['prob_C_1.5'], bins=50, alpha=0.5, label='Model C (LSTM)')
    plt.hist(sim_df['prob_D_1.5'], bins=50, alpha=0.5, label='Model D (LightGBM)')
    plt.hist(sim_df['prob_E_1.5'], bins=50, alpha=0.5, label='Model E (MLP)')
    plt.axvline(x=0.5, color='k', linestyle='--', label='Kararsızlık Sınırı (0.5)')
    plt.axvline(x=0.65, color='r', linestyle='--', label='Hedef Eşik (0.65)')
    plt.legend()
    
    # 3.0x Olasılıkları
    plt.subplot(2, 1, 2)
    plt.title("3.0x Üstü Olasılık Dağılımı")
    plt.hist(sim_df['prob_A_3.0'], bins=50, alpha=0.5, label='Model A (CatBoost)')
    plt.hist(sim_df['prob_C_3.0'], bins=50, alpha=0.5, label='Model C (LSTM)')
    plt.hist(sim_df['prob_D_3.0'], bins=50, alpha=0.5, label='Model D (LightGBM)')
    plt.hist(sim_df['prob_E_3.0'], bins=50, alpha=0.5, label='Model E (MLP)')
    plt.axvline(x=0.5, color='k', linestyle='--', label='Kararsızlık Sınırı (0.5)')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
else:
    print("Simülasyon için yeterli veri yok.")

In [ ]:
# 10. Modelleri İndir
!zip -r models.zip models/
from google.colab import files
files.download('models.zip')